<a href="https://colab.research.google.com/github/priyu9-star/BudgetWise-AI-based-Expense-Forecasting-Tool-Batch-6-Team-C-/blob/main/SpendGenie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ======================================================
# SPENDGENIE - Complete AI-Powered Personal Expense Tracker
# With Landing Page, Hamburger Menu, Theme Toggle, and All Features
# ======================================================

# Install dependencies
!pip install -q flask flask_sqlalchemy flask_bcrypt pyngrok pandas plotly scikit-learn numpy

import os, io, uuid, shutil, json
from pathlib import Path
from datetime import datetime, timedelta
from flask import Flask, render_template, render_template_string, request, redirect, session, jsonify, send_file, url_for
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

# ----------------- Configuration -----------------
NGROK_TOKEN = "36I83QjZdc3Re5qRgjmaBGUOuqD_7tBbqCsW1a2ndABreuJzA"
DEFAULT_DATA_PATH = "/mnt/data/Personal_Finance_Dataset.csv"
BASE_DIR = Path("/tmp/spendgenie_app")
UPLOAD_ROOT = BASE_DIR / "uploads"
TEMPLATE_DIR = BASE_DIR / "templates"
STATIC_DIR = BASE_DIR / "static"
for d in (BASE_DIR, UPLOAD_ROOT, TEMPLATE_DIR, STATIC_DIR):
    d.mkdir(parents=True, exist_ok=True)

# ----------------- Flask app -----------------
app = Flask(__name__, template_folder=str(TEMPLATE_DIR), static_folder=str(STATIC_DIR))
app.secret_key = "spendgenie-secret-key-2025"
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///" + str(BASE_DIR / "users.db")
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

# ----------------- Database model -----------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80))
    email = db.Column(db.String(120), unique=True)
    password = db.Column(db.String(200))

    def check_pass(self, pwd):
        return bcrypt.check_password_hash(self.password, pwd)

with app.app_context():
    db.create_all()

# ----------------- Utilities -----------------
def safe_session_id():
    sid = session.get("session_id")
    if not sid:
        sid = str(uuid.uuid4())
        session["session_id"] = sid
    return sid

def user_upload_dir():
    sid = safe_session_id()
    d = UPLOAD_ROOT / sid
    d.mkdir(parents=True, exist_ok=True)
    return d

def save_uploaded_csv(file_storage):
    updir = user_upload_dir()
    filename = "uploaded.csv"
    path = updir / filename
    file_storage.save(str(path))
    return str(path)

def get_user_csv_path():
    updir = user_upload_dir()
    path = updir / "uploaded.csv"
    if path.exists():
        return str(path)
    if os.path.exists(DEFAULT_DATA_PATH):
        try:
            shutil.copy(DEFAULT_DATA_PATH, str(path))
            return str(path)
        except Exception:
            return None
    return None

def guess_col(candidates, columns):
    cols_lower = {c.lower(): c for c in columns}
    for name in candidates:
        if name.lower() in cols_lower:
            return cols_lower[name.lower()]
    for name in candidates:
        for c in columns:
            if name.lower() in c.lower():
                return c
    return None

def clean_amount_val(x):
    try:
        if pd.isna(x):
            return np.nan
        s = str(x).strip()
        if s == "":
            return np.nan
        if s.startswith("(") and s.endswith(")"):
            s = "-" + s[1:-1]
        s = s.replace(",", "")
        s2 = "".join(ch for ch in s if ch.isdigit() or ch in ".-+")
        if s2 in ("", ".", "+", "-", "+.", "-."):
            return np.nan
        return float(s2)
    except:
        return np.nan

def load_and_prepare(csv_path):
    df = pd.read_csv(csv_path, low_memory=False)
    df.columns = [c.strip() for c in df.columns]
    cols = df.columns.tolist()
    date_col = guess_col(["date","transaction_date","txn_date","posted_date","timestamp"], cols)
    amt_col = guess_col(["amount","amt","value","transaction_amount","debit","credit"], cols)
    cat_col = guess_col(["category","cat","expense_category","merchant_category","type","merchant"], cols)
    desc_col = guess_col(["description","memo","narration","details"], cols)
    if not date_col or not amt_col:
        raise ValueError("Could not detect date and amount columns. Ensure CSV has date & amount columns.")
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col])
    df["_raw_amount_"] = df[amt_col]
    df["amount_clean"] = df[amt_col].map(clean_amount_val)
    df = df.dropna(subset=["amount_clean"])
    if cat_col:
        df["category_clean"] = df[cat_col].fillna("Uncategorized").astype(str)
    else:
        df["category_clean"] = "Uncategorized"
    if desc_col:
        df["description_clean"] = df[desc_col].astype(str)
    else:
        df["description_clean"] = ""
    df["year"] = df[date_col].dt.year
    df["month"] = df[date_col].dt.month
    df["month_year"] = df[date_col].dt.to_period("M").astype(str)
    df["day"] = df[date_col].dt.date
    nonzero = df[df["amount_clean"] != 0]["amount_clean"]
    neg_frac = (nonzero < 0).mean() if len(nonzero) > 0 else 0
    if neg_frac >= 0.5:
        df["expense"] = df["amount_clean"].apply(lambda v: -v if v < 0 else 0.0)
        df["income"] = df["amount_clean"].apply(lambda v: v if v > 0 else 0.0)
        sign_rule = "expenses_negative"
    else:
        type_col = guess_col(["type","transaction_type","kind"], cols)
        if type_col and type_col in df.columns:
            t = df[type_col].astype(str).str.lower()
            exp_mask = t.isin(["expense","debit","withdrawal","payment","spent"])
            df["expense"] = df["amount_clean"].where(exp_mask, 0).abs()
            df["income"] = df["amount_clean"].where(~exp_mask, 0).abs()
            sign_rule = "type_based"
        else:
            df["expense"] = df["amount_clean"].apply(lambda v: v if v > 0 else 0.0)
            df["income"] = df["amount_clean"].apply(lambda v: -v if v < 0 else 0.0)
            sign_rule = "positive_expense"
    return df, {"date_col": date_col, "amt_col": amt_col, "cat_col": cat_col, "sign_rule": sign_rule}

# ----------------- Simple Prediction Functions -----------------
def normalize_main_df():
    """Normalize main dataframe for simple predictions"""
    global main_df, available_categories
    if main_df is None or main_df.empty:
        main_df = pd.DataFrame(columns=["item", "price", "date", "description"])
    for c in ["item", "price", "date", "description"]:
        if c not in main_df.columns:
            main_df[c] = None
    # price numeric
    main_df["price"] = pd.to_numeric(main_df["price"], errors="coerce").fillna(0.0)
    # parse dates robustly
    def parse_date(x):
        if pd.isna(x) or x == "":
            return pd.NaT
        if isinstance(x, (pd.Timestamp, datetime)):
            return pd.to_datetime(x)
        try:
            return pd.to_datetime(x)
        except:
            for fmt in ("%Y-%m-%d","%d-%m-%Y","%d/%m/%Y","%m/%d/%Y"):
                try:
                    return pd.to_datetime(x, format=fmt)
                except:
                    pass
            return pd.NaT
    main_df["date"] = main_df["date"].apply(parse_date)
    available_categories = sorted(main_df["item"].dropna().astype(str).unique().tolist())

def totals_by_category(period="all"):
    """Return dict category -> total for the given period (all,daily,weekly,monthly)."""
    normalize_main_df()
    df = main_df.copy()
    if df.empty:
        return {}
    df = df.dropna(subset=["item","price"]).copy()
    if period == "daily":
        today = pd.Timestamp(datetime.now().date())
        df = df[df["date"].dt.date == today.date()]
    elif period == "weekly":
        today = pd.Timestamp(datetime.now().date())
        week_ago = today - pd.Timedelta(days=6)  # 7-day window
        df = df[df["date"].dt.date >= week_ago.date()]
    elif period == "monthly":
        today = pd.Timestamp(datetime.now().date())
        month_ago = today - pd.Timedelta(days=29)  # 30-day window
        df = df[df["date"].dt.date >= month_ago.date()]
    grouped = df.groupby("item")["price"].sum().to_dict()
    # ensure keys for all known categories
    for cat in available_categories:
        grouped.setdefault(cat, 0.0)
    return grouped

def total_expense(period="all"):
    return sum(totals_by_category(period).values())

# Global variables for simple expense tracking
main_df = pd.DataFrame(columns=["item", "price", "date", "description"])
saved_expenses = []
available_categories = []

# ----------------- Advanced Prediction Functions -----------------
def aggregate_past_months(exp_df):
    """Aggregate expenses by month"""
    exp_df = exp_df.copy()
    exp_df['year'] = exp_df['date'].dt.year
    exp_df['month'] = exp_df['date'].dt.month

    tmp = exp_df.groupby(['year', 'month'], as_index=False)['amount'].sum()
    tmp['label'] = tmp.apply(
        lambda r: pd.Timestamp(year=int(r['year']), month=int(r['month']), day=1).strftime('%b %Y'),
        axis=1
    )
    tmp = tmp.sort_values(['year', 'month']).reset_index(drop=True)
    return tmp[['label', 'amount']]

def predict_next_total(monthly_totals_df):
    """Predict next month's total using linear regression"""
    if len(monthly_totals_df) == 0:
        return 0.0
    if len(monthly_totals_df) == 1:
        return float(monthly_totals_df['amount'].iloc[0])

    monthly_totals_df = monthly_totals_df.copy().reset_index(drop=True)
    monthly_totals_df['Month_Index'] = np.arange(1, len(monthly_totals_df)+1)
    X = monthly_totals_df[['Month_Index']].values
    y = monthly_totals_df['amount'].values

    try:
        model = LinearRegression().fit(X, y)
        next_idx = np.array([[monthly_totals_df['Month_Index'].max() + 1]])
        pred = float(model.predict(next_idx)[0])
        return round(max(pred, 0.0), 2)
    except:
        # Fallback: return average of last 3 months
        last_3_avg = monthly_totals_df['amount'].tail(3).mean()
        return round(float(last_3_avg), 2)

def predict_next_by_category(df_exp):
    """Predict next month's expenses by category"""
    df = df_exp.copy()
    df['Year'] = df['date'].dt.year
    df['Month'] = df['date'].dt.month

    if len(df) == 0:
        return {}

    df['ym'] = df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2)
    months_sorted = sorted(df['ym'].unique())
    month_map = {m: i+1 for i, m in enumerate(months_sorted)}
    df['Month_Index'] = df['ym'].map(month_map)
    next_month_idx = max(month_map.values()) + 1 if month_map else 1

    grouped = df.groupby(['category','Month_Index'], as_index=False)['amount'].sum()
    categories = grouped['category'].unique().tolist()
    preds = {}

    for cat in categories:
        cat_df = grouped[grouped['category'] == cat].sort_values('Month_Index')
        X = cat_df[['Month_Index']].values
        y = cat_df['amount'].values

        if len(X) == 0:
            preds[cat] = 0.0
            continue
        if len(X) == 1:
            preds[cat] = round(float(y[0]), 2)
            continue

        try:
            model = LinearRegression().fit(X, y)
            pred = float(model.predict([[next_month_idx]])[0])
            preds[cat] = round(max(pred, 0.0), 2)
        except:
            last_avg = cat_df['amount'].tail(3).mean()
            preds[cat] = round(float(last_avg), 2)

    return preds

# ----------------- Landing Page Template -----------------
landing_template = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <title>SpendGenie - AI Expense Forecasting</title>

  <style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;900&display=swap');

    :root {
      --button-radius: 8px; /* Default button shape */
    }

    * {
      margin: 0;
      padding: 0;
      box-sizing: border-box;
      font-family: 'Inter', sans-serif;
    }

    body {
      background: linear-gradient(to bottom, #ff4f8c, #ffffff);
      min-height: 100vh;
      display: flex;
      flex-direction: column;
      align-items: center;
      color: #333;
    }

    /* NAVBAR */
    .navbar {
      width: 100%;
      padding: 25px 60px;
      display: flex;
      align-items: center;
      justify-content: space-between;
    }

    .logo {
      display: flex;
      align-items: center;
      gap: 10px;
      cursor: pointer;
    }

    .logo-circle {
      width: 40px;
      height: 40px;
      background: #ddd;
      border-radius: 50%;
    }

    .menu {
      display: flex;
      gap: 35px;
      font-size: 18px;
    }

    .menu a {
      text-decoration: none;
      color: #fff;
      font-weight: 500;
    }

    .nav-buttons {
      display: flex;
      gap: 15px;
    }

    .btn-outline {
      padding: 8px 18px;
      border: 1.5px solid #fff;
      border-radius: var(--button-radius);
      background: transparent;
      color: #fff;
      cursor: pointer;
      transition: 0.2s;
    }

    .btn-outline:hover {
      background: rgba(255,255,255,0.2);
    }

    .btn-fill {
      padding: 8px 18px;
      border-radius: var(--button-radius);
      background: #5a2aff;
      color: #fff;
      cursor: pointer;
      border: none;
      transition: 0.2s;
    }

    /* HERO SECTION */
    .hero {
      text-align: center;
      margin-top: 30px;
      max-width: 800px;
    }

    .hero h1 {
      font-size: 40px;
      font-weight: 800;
      color: #fff;
      line-height: 1.3;
    }

    .hero p {
      margin-top: 15px;
      font-size: 18px;
      color: #f5f5f5;
      line-height: 1.5;
    }

    .get-started {
      margin-top: 25px;
      padding: 12px 28px;
      font-size: 18px;
      border-radius: var(--button-radius);
      background: #5a2aff;
      border: none;
      color: #fff;
      cursor: pointer;
      transition: 0.2s;
    }

    /* LAPTOP IMAGE */
    .laptop-container {
      margin-top: 40px;
      width: 900px;
    }

    .laptop-container img {
      width: 100%;
      border-radius: 10px;
      box-shadow: 0px 10px 25px rgba(0,0,0,0.25);
    }

    /* FOOTER */
    footer {
      width: 100%;
      background: #222;
      color: #fff;
      padding: 30px 0;
      margin-top: 60px;
      text-align: center;
      font-size: 16px;
    }

    footer a {
      color: #ccc;
      margin: 0 12px;
      text-decoration: none;
      font-size: 15px;
    }

    footer a:hover {
      color: #fff;
    }

    /* Responsive Design */
    @media (max-width: 1024px) {
      .laptop-container {
        width: 700px;
      }
    }

    @media (max-width: 768px) {
      .navbar {
        padding: 20px;
        flex-direction: column;
        gap: 20px;
      }

      .menu {
        gap: 20px;
        font-size: 16px;
      }

      .hero h1 {
        font-size: 32px;
      }

      .hero p {
        font-size: 16px;
        padding: 0 20px;
      }

      .laptop-container {
        width: 90%;
      }
    }

    @media (max-width: 480px) {
      .hero h1 {
        font-size: 28px;
      }

      .nav-buttons {
        flex-direction: column;
        width: 100%;
      }

      .btn-outline, .btn-fill {
        width: 100%;
        text-align: center;
      }
    }

  </style>
</head>

<body>

  <!-- NAVBAR -->
  <div class="navbar">
    <div class="logo">
      <div class="logo-circle"></div>
      <span style="font-size: 20px; color:#fff; font-weight:600;">SpendGenie </span>
    </div>

    <div class="menu">
      <a href="#">About</a>
      <a href="#">Contact Us</a>
    </div>

    <div class="nav-buttons">
      <button class="btn-outline">Sign In</button>
      <button class="btn-fill">Register</button>
    </div>
  </div>

  <!-- HERO CONTENT -->
  <div class="hero">
    <h1>Your Budget, Re-imagined With<br>Intelligence with SpendGenie</h1>

    <p>
      An AI that listens to the rhythm of your spending and predicts tomorrow's trends.<br>
      Transform financial chaos into crystal-clear insights.
    </p>

    <button class="get-started">Get Started</button>
  </div>

  <!-- MAIN DASHBOARD MOCKUP -->
  <div class="laptop-container">
    <img src="https://image2url.com/images/1763729644845-c1e18bd1-2559-4901-b201-9b4da762f269.png" alt="Dashboard Mockup">
  </div>

  <!-- FOOTER -->
  <footer>
    <p>© 2025 SpendGenie | All Rights Reserved</p>
    <div style="margin-top:12px;">
      <a href="#">About</a>
      <a href="#">Contact Us</a>
      <a href="#">Privacy Policy</a>
    </div>
  </footer>

  <script>
    // Make buttons functional
    document.querySelector('.btn-outline').addEventListener('click', function() {
      window.location.href = "/login";
    });

    document.querySelector('.btn-fill').addEventListener('click', function() {
      window.location.href = "/register";
    });

    document.querySelector('.get-started').addEventListener('click', function() {
      window.location.href = "/register";
    });

    // Make logo clickable to go home
    document.querySelector('.logo').addEventListener('click', function() {
      window.location.href = "/";
    });

    // Make menu links functional (update hrefs as needed)
    document.querySelectorAll('.menu a')[0].addEventListener('click', function(e) {
      e.preventDefault();
      // Add your about page logic here
      alert('About page coming soon!');
    });

    document.querySelectorAll('.menu a')[1].addEventListener('click', function(e) {
      e.preventDefault();
      // Add your contact page logic here
      alert('Contact page coming soon!');
    });
  </script>
</body>
</html>"""

# ----------------- App Base Template with Sidebar -----------------
app_base_template = """<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>SpendGenie - {{ page_title }}</title>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<style>
    :root {
        --bg-dark: #0f0f0f;
        --bg-light: #f8f9fa;
        --sidebar-dark: #151515;
        --sidebar-light: #ffffff;
        --card-dark: #1c1c1c;
        --card-light: #ffffff;
        --text-dark: #ffffff;
        --text-light: #333333;
        --accent: #0ea5ff;
        --muted: #b0b0b0;
        --border-dark: rgba(255,255,255,0.1);
        --border-light: rgba(0,0,0,0.1);
    }

    * {
        margin: 0;
        padding: 0;
        box-sizing: border-box;
        font-family: 'Inter', sans-serif;
        transition: background-color 0.3s ease, color 0.3s ease, border-color 0.3s ease;
    }

    body {
        margin: 0;
        background: var(--bg-dark);
        color: var(--text-dark);
        font-family: Inter, sans-serif;
        display: flex;
        transition: background 0.3s, color 0.3s;
        min-height: 100vh;
    }

    body.light-mode {
        background: var(--bg-light);
        color: var(--text-light);
    }

    /* Sidebar */
    .sidebar {
        width: 240px;
        background: var(--sidebar-dark);
        height: 100vh;
        border-right: 1px solid var(--border-dark);
        padding-top: 20px;
        position: fixed;
        top: 0;
        left: 0;
        transition: all 0.3s ease;
        overflow-x: hidden;
        overflow-y: auto;
        z-index: 1000;
    }

    .light-mode .sidebar {
        background: var(--sidebar-light);
        border-right: 1px solid var(--border-light);
    }

    .sidebar.collapsed {
        width: 70px;
        overflow: hidden;
    }

    /* Hamburger Menu Button - Improved */
    .toggle-btn {
        position: absolute;
        top: 15px;
        right: -15px;
        width: 30px;
        height: 30px;
        background: var(--accent);
        border-radius: 50%;
        color: white;
        font-size: 16px;
        border: none;
        cursor: pointer;
        display: flex;
        align-items: center;
        justify-content: center;
        z-index: 1001;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.2);
        transition: all 0.3s ease;
    }

    .toggle-btn:hover {
        transform: scale(1.1);
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.3);
    }

    .toggle-btn:active {
        transform: scale(0.95);
    }

    /* Mobile Hamburger for Small Screens */
    .mobile-hamburger {
        display: none;
        position: fixed;
        top: 15px;
        left: 15px;
        width: 40px;
        height: 40px;
        background: var(--accent);
        border-radius: 10px;
        color: white;
        font-size: 20px;
        border: none;
        cursor: pointer;
        z-index: 1002;
        align-items: center;
        justify-content: center;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.2);
    }

    .sidebar-title {
        font-size: 22px;
        font-weight: 700;
        margin-left: 25px;
        margin-bottom: 35px;
        opacity: 1;
        transition: opacity 0.3s ease;
        display: flex;
        align-items: center;
        gap: 8px;
        color: var(--accent);
        white-space: nowrap;
    }

    .sidebar.collapsed .sidebar-title {
        opacity: 0;
        pointer-events: none;
    }

    /* Menu Items */
    .menu-item {
        padding: 12px 20px;
        display: flex;
        align-items: center;
        gap: 12px;
        font-size: 16px;
        cursor: pointer;
        border-radius: 10px;
        margin: 8px 10px;
        color: var(--muted);
        white-space: nowrap;
        text-decoration: none;
        transition: all 0.3s;
    }

    .light-mode .menu-item {
        color: #666;
    }

    .menu-item:hover {
        background: rgba(14, 165, 255, 0.15);
        color: var(--accent);
        transform: translateX(5px);
    }

    .light-mode .menu-item:hover {
        background: rgba(14, 165, 255, 0.08);
    }

    .menu-item.active {
        background: linear-gradient(135deg, #0ea5ff, #5a2aff);
        color: white;
        box-shadow: 0 4px 12px rgba(14, 165, 255, 0.3);
    }

    .menu-label {
        transition: all 0.3s ease;
        font-weight: 500;
        opacity: 1;
    }

    .sidebar.collapsed .menu-label {
        opacity: 0;
        width: 0;
        overflow: hidden;
    }

    .icon {
        font-size: 18px;
        min-width: 24px;
        text-align: center;
        transition: transform 0.3s ease;
    }

    .menu-item:hover .icon {
        transform: scale(1.1);
    }

    .sidebar-bottom {
        position: absolute;
        bottom: 25px;
        width: 100%;
        padding-bottom: 20px;
    }

    /* Main Content Area */
    .main {
        margin-left: 240px;
        padding: 30px 50px;
        width: calc(100% - 240px);
        transition: all 0.3s ease;
        min-height: 100vh;
    }

    .sidebar.collapsed ~ .main {
        margin-left: 70px;
        width: calc(100% - 70px);
    }

    /* Overlay for mobile when sidebar is open */
    .sidebar-overlay {
        display: none;
        position: fixed;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background: rgba(0, 0, 0, 0.5);
        z-index: 999;
        backdrop-filter: blur(3px);
    }

    /* Content Styles */
    .main-container{max-width:1160px;margin:0 auto;padding:0 18px}
    .filters{display:flex;gap:12px;flex-wrap:wrap;align-items:center;justify-content:center;margin-bottom:12px;padding:12px;background:rgba(14,165,255,0.05);border-radius:10px}
    .light-mode .filters{background:rgba(14,165,255,0.02)}
    .filters select,.filters input{padding:10px;border-radius:8px;border:1px solid var(--border-dark);background:rgba(255,255,255,0.02);color:inherit}
    .light-mode .filters select,.light-mode .filters input{border:1px solid var(--border-light);background:rgba(0,0,0,0.02)}
    button{background:var(--accent);color:white;border:none;padding:10px 12px;border-radius:8px;cursor:pointer;transition:all 0.3s}
    button:hover{opacity:0.95;transform:translateY(-1px)}
    .ai-summary{background:rgba(14,165,255,0.05);padding:14px;border-radius:12px;margin:10px 0;text-align:center}
    .light-mode .ai-summary{background:rgba(14,165,255,0.02)}
    .grid{display:grid;grid-template-columns:repeat(2,1fr);gap:18px}
    @media (max-width: 768px) { .grid{grid-template-columns:1fr} }
    .card{background:var(--card-dark);padding:14px;border-radius:12px;min-height:220px;border:1px solid var(--border-dark)}
    .light-mode .card{background:var(--card-light);border:1px solid var(--border-light)}
    .empty{color:#93c5fd;text-align:center;padding:30px}
    .auth-box{width:360px;margin:60px auto;padding:30px;background:rgba(255,255,255,0.02);border-radius:14px;text-align:center}
    .light-mode .auth-box{background:rgba(0,0,0,0.02)}
    .auth-box input{width:100%;padding:12px;margin:8px 0;border-radius:8px;border:1px solid var(--border-dark);background:rgba(255,255,255,0.01);color:inherit}
    .light-mode .auth-box input{border:1px solid var(--border-light);background:rgba(0,0,0,0.01)}
    .auth-box button{width:100%;padding:12px;border-radius:8px;border:none;background:var(--accent);color:white}
    .preview{width:100%;border-collapse:collapse}
    .preview th,.preview td{padding:6px;border-bottom:1px solid var(--border-dark)}
    .light-mode .preview th,.light-mode .preview td{border-bottom:1px solid var(--border-light)}
    .container-center{max-width:900px;margin:40px auto;padding:18px;text-align:center}
    .error{color:#ff6b6b}
    .upload-box{background:var(--card-dark);padding:28px;border-radius:14px;border:1px solid var(--border-dark);text-align:center}
    .light-mode .upload-box{background:var(--card-light);border:1px solid var(--border-light)}

    /* Header with Theme Toggle */
    .page-header {
        display: flex;
        justify-content: space-between;
        align-items: center;
        margin-bottom: 30px;
        padding-bottom: 20px;
        border-bottom: 1px solid var(--border-dark);
    }

    .light-mode .page-header {
        border-bottom: 1px solid var(--border-light);
    }

    .page-header h1 {
        font-size: 32px;
        background: linear-gradient(135deg, #0ea5ff, #5a2aff);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
    }

    .header-controls {
        display: flex;
        gap: 15px;
        align-items: center;
    }

    .theme-toggle-btn {
        background: none;
        border: none;
        font-size: 20px;
        cursor: pointer;
        color: inherit;
        padding: 8px;
        border-radius: 50%;
        transition: background 0.3s;
    }

    .theme-toggle-btn:hover {
        background: rgba(255, 255, 255, 0.1);
    }

    .light-mode .theme-toggle-btn:hover {
        background: rgba(0, 0, 0, 0.05);
    }

    /* Chat Widget */
    .chat-fab {
        position: fixed;
        right: 22px;
        bottom: 22px;
        background: linear-gradient(135deg, #0ea5ff, #5a2aff);
        width: 60px;
        height: 60px;
        border-radius: 16px;
        display: grid;
        place-items: center;
        color: white;
        font-weight: 700;
        cursor: pointer;
        box-shadow: 0 10px 30px rgba(90,42,255,0.3);
        z-index: 99;
        transition: transform 0.3s;
    }

    .chat-fab:hover {
        transform: scale(1.1);
    }

    .chat-window {
        position: fixed;
        right: 22px;
        bottom: 96px;
        width: 360px;
        border-radius: 12px;
        background: var(--card-dark);
        color: var(--text-dark);
        box-shadow: 0 30px 60px rgba(0,0,0,0.6);
        overflow: hidden;
        display: none;
        flex-direction: column;
        z-index: 100;
        border: 1px solid var(--border-dark);
    }

    .light-mode .chat-window {
        background: var(--card-light);
        border: 1px solid var(--border-light);
        box-shadow: 0 30px 60px rgba(0,0,0,0.1);
    }

    .chat-header {
        padding: 12px 14px;
        background: linear-gradient(90deg, #041224, #063b66);
        display: flex;
        align-items: center;
        justify-content: space-between;
    }

    .chat-body {
        padding: 12px;
        max-height: 320px;
        overflow: auto;
    }

    .chat-input {
        display: flex;
        padding: 10px;
        border-top: 1px solid var(--border-dark);
    }

    .light-mode .chat-input {
        border-top: 1px solid var(--border-light);
    }

    .chat-input input {
        flex: 1;
        padding: 10px;
        border-radius: 10px;
        border: 1px solid var(--border-dark);
        background: rgba(255,255,255,0.02);
        color: inherit;
        margin-right: 8px;
    }

    .light-mode .chat-input input {
        border: 1px solid var(--border-light);
        background: rgba(0,0,0,0.02);
    }

    /* Responsive adjustments */
    @media (max-width: 1024px) {
        .sidebar {
            width: 200px;
        }

        .main {
            margin-left: 200px;
            width: calc(100% - 200px);
            padding: 25px;
        }

        .sidebar.collapsed ~ .main {
            margin-left: 70px;
            width: calc(100% - 70px);
        }
    }

    @media (max-width: 768px) {
        .mobile-hamburger {
            display: flex;
        }

        .sidebar {
            width: 280px;
            left: -280px;
            box-shadow: 5px 0 25px rgba(0, 0, 0, 0.3);
            z-index: 1000;
        }

        .sidebar.mobile-open {
            left: 0;
        }

        .main {
            margin-left: 0;
            width: 100%;
            padding: 70px 20px 20px;
        }

        .sidebar-overlay.active {
            display: block;
        }

        .toggle-btn {
            display: none;
        }

        .sidebar-title {
            margin-top: 20px;
        }

        .page-header {
            flex-direction: column;
            align-items: flex-start;
            gap: 15px;
        }

        .header-controls {
            width: 100%;
            justify-content: space-between;
        }

        .chat-window {
            width: calc(100% - 40px);
            right: 20px;
            bottom: 90px;
        }

        .chat-fab {
            right: 20px;
            bottom: 20px;
        }
    }
</style>
</head>
<body id="bodyRoot">
    <!-- Mobile Hamburger Button -->
    <button class="mobile-hamburger" id="mobileHamburger">
        ☰
    </button>

    <!-- Overlay for mobile -->
    <div class="sidebar-overlay" id="sidebarOverlay"></div>

    <!-- SIDEBAR -->
    <div class="sidebar" id="sidebar">
        <button class="toggle-btn" onclick="toggleSidebar()">☰</button>
        <div class="sidebar-title">⚡ SpendGenie</div>

        <a class="menu-item {% if active_page == 'upload' %}active{% endif %}" href="/upload">
            <span class="icon">📁</span><span class="menu-label">Upload CSV</span>
        </a>
        <a class="menu-item {% if active_page == 'dashboard' %}active{% endif %}" href="/dashboard">
            <span class="icon">📊</span><span class="menu-label">Dashboard</span>
        </a>
        <a class="menu-item {% if active_page == 'prediction' %}active{% endif %}" href="/prediction">
            <span class="icon">🤖</span><span class="menu-label">Prediction</span>
        </a>
        <a class="menu-item {% if active_page == 'expense_input' %}active{% endif %}" href="/expense_input">
            <span class="icon">📝</span><span class="menu-label">Add Expense</span>
        </a>

        <div class="sidebar-bottom">
            <a class="menu-item {% if active_page == 'profile' %}active{% endif %}" href="/profile">
                <span class="icon">👤</span><span class="menu-label">Profile</span>
            </a>
            <a class="menu-item {% if active_page == 'logout' %}active{% endif %}" href="/logout">
                <span class="icon">🚪</span><span class="menu-label">Logout</span>
            </a>
            <a class="menu-item" href="/" style="margin-top: 20px;">
                <span class="icon">🏠</span><span class="menu-label">Home</span>
            </a>
        </div>
    </div>

    <!-- MAIN CONTENT -->
    <div class="main">
        <div class="page-header">
            <h1>{{ page_title }}</h1>
            <div class="header-controls">
                <button class="theme-toggle-btn" id="themeToggle">
                    🌓
                </button>
            </div>
        </div>

        {{ content | safe }}
    </div>

    <!-- Chat Widget -->
    <div class="chat-fab" id="chatFab">💬</div>
    <div id="chatWindow" class="chat-window">
        <div class="chat-header">
            <div>SpendGenie AI Assistant</div>
            <div style="opacity:0.9; font-size:13px; cursor:pointer" id="closeChat">✕</div>
        </div>
        <div class="chat-body" id="chatBody"></div>
        <div class="chat-input">
            <input id="chatInput" placeholder="Ask for suggestions, e.g. 'How can I reduce dining spend?'" />
            <button id="sendChat">Send</button>
        </div>
    </div>

    <script>
    // Sidebar Toggle
    function toggleSidebar() {
        const sidebar = document.getElementById('sidebar');
        sidebar.classList.toggle('collapsed');

        // Save sidebar state
        const isCollapsed = sidebar.classList.contains('collapsed');
        localStorage.setItem('sidebarCollapsed', isCollapsed);
    }

    // Theme Toggle
    const themeToggle = document.getElementById('themeToggle');
    const body = document.getElementById('bodyRoot');

    // Check for saved theme preference - Use consistent key
    let savedTheme = localStorage.getItem('theme') || localStorage.getItem('appTheme');
    if (savedTheme === 'light') {
        body.classList.add('light-mode');
        themeToggle.textContent = '🌙';
    }

    themeToggle.addEventListener('click', () => {
        body.classList.toggle('light-mode');
        if (body.classList.contains('light-mode')) {
            localStorage.setItem('theme', 'light');
            localStorage.setItem('appTheme', 'light'); // Sync both
            themeToggle.textContent = '🌙';
        } else {
            localStorage.setItem('theme', 'dark');
            localStorage.setItem('appTheme', 'dark'); // Sync both
            themeToggle.textContent = '🌓';
        }
    });

    // Mobile Hamburger Menu
    const mobileHamburger = document.getElementById('mobileHamburger');
    const sidebarOverlay = document.getElementById('sidebarOverlay');
    const sidebar = document.getElementById('sidebar');

    // Check screen size and initialize
    function checkScreenSize() {
        if (window.innerWidth <= 768) {
            // Mobile: Check saved sidebar state
            const savedState = localStorage.getItem('sidebarCollapsed');
            if (savedState === 'true') {
                sidebar.classList.add('collapsed');
            }
        } else {
            // Desktop: Remove mobile open class
            sidebar.classList.remove('mobile-open');
            sidebarOverlay.classList.remove('active');
        }
    }

    mobileHamburger.addEventListener('click', () => {
        sidebar.classList.toggle('mobile-open');
        sidebarOverlay.classList.toggle('active');
    });

    sidebarOverlay.addEventListener('click', () => {
        sidebar.classList.remove('mobile-open');
        sidebarOverlay.classList.remove('active');
    });

    // Close sidebar when clicking a menu item on mobile
    document.querySelectorAll('.menu-item').forEach(item => {
        item.addEventListener('click', () => {
            if (window.innerWidth <= 768) {
                sidebar.classList.remove('mobile-open');
                sidebarOverlay.classList.remove('active');
            }
        });
    });

    // Load saved sidebar state on page load
    window.addEventListener('DOMContentLoaded', () => {
        checkScreenSize();

        // Load saved sidebar state (for desktop)
        if (window.innerWidth > 768) {
            const savedState = localStorage.getItem('sidebarCollapsed');
            if (savedState === 'true') {
                sidebar.classList.add('collapsed');
            }
        }
    });

    // Check screen size on resize
    window.addEventListener('resize', checkScreenSize);

    // Chat functionality
    const chatFab = document.getElementById('chatFab');
    const chatWindow = document.getElementById('chatWindow');
    const closeChat = document.getElementById('closeChat');
    const chatBody = document.getElementById('chatBody');
    const chatInput = document.getElementById('chatInput');
    const sendChat = document.getElementById('sendChat');

    chatFab.addEventListener('click', ()=>{
        chatWindow.style.display='flex';
        chatBody.scrollTop = chatBody.scrollHeight;
    });
    closeChat.addEventListener('click', ()=>{ chatWindow.style.display='none'; });

    function addMessage(text, who='bot'){
        const div = document.createElement('div');
        div.style.marginBottom='10px';
        div.style.padding='8px';
        div.style.borderRadius='10px';
        div.style.maxWidth='88%';
        if(who==='user'){
            div.style.marginLeft='auto';
            div.style.background='linear-gradient(90deg,#2b2b2b,#1a1a1a)';
            div.innerText = text;
        } else {
            div.style.background='linear-gradient(90deg,#063b66,#0b2a4a)';
            div.innerText = text;
        }
        chatBody.appendChild(div);
        chatBody.scrollTop = chatBody.scrollHeight;
    }

    async function sendToServer(msg){
        addMessage(msg,'user');
        try{
            const res = await fetch('/chat', {
                method:'POST',
                headers:{'Content-Type':'application/json'},
                body: JSON.stringify({message: msg})
            });
            const j = await res.json();
            addMessage(j.reply,'bot');
        }catch(e){
            addMessage("Sorry — chat failed. "+e,'bot');
        }
    }

    sendChat.addEventListener('click', ()=>{
        const v = chatInput.value.trim();
        if(!v) return;
        sendToServer(v);
        chatInput.value='';
    });

    chatInput.addEventListener('keydown', (e)=>{
        if(e.key==='Enter'){ sendChat.click(); }
    });

    // Add welcome message
    addMessage("Hello! I'm your SpendGenie AI assistant. Ask me about your spending patterns, predictions, or budgeting tips.");
    </script>
</body>
</html>"""

# ----------------- Auth templates -----------------
login_html = """<!doctype html>
<html><head><meta charset="utf-8"><title>Login - SpendGenie</title>
<style>
    :root {
        --bg-dark: #0f0f0f;
        --bg-light: #f8f9fa;
        --card-dark: #1c1c1c;
        --card-light: #ffffff;
        --text-dark: #ffffff;
        --text-light: #333333;
        --accent: #0ea5ff;
        --muted: #b0b0b0;
    }

    * {
        margin: 0;
        padding: 0;
        box-sizing: border-box;
        font-family: 'Inter', sans-serif;
        transition: background-color 0.3s ease, color 0.3s ease, border-color 0.3s ease;
    }

    body {
        margin: 0;
        background: var(--bg-dark);
        color: var(--text-dark);
        min-height: 100vh;
        display: flex;
        align-items: center;
        justify-content: center;
        transition: background 0.3s, color 0.3s;
    }

    body.light-mode {
        background: var(--bg-light);
        color: var(--text-light);
    }

    .auth-container {
        width: 100%;
        max-width: 400px;
        padding: 20px;
    }

    .auth-box {
        background: var(--card-dark);
        padding: 40px;
        border-radius: 16px;
        text-align: center;
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.3);
        border: 1px solid rgba(255, 255, 255, 0.1);
    }

    .light-mode .auth-box {
        background: var(--card-light);
        border: 1px solid rgba(0, 0, 0, 0.1);
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
    }

    .auth-box h2 {
        font-size: 28px;
        margin-bottom: 10px;
        color: var(--accent);
    }

    .auth-box p {
        color: var(--muted);
        margin-bottom: 30px;
    }

    .light-mode .auth-box p {
        color: #666;
    }

    .auth-box input {
        width: 100%;
        padding: 14px;
        margin: 10px 0;
        border-radius: 10px;
        border: 1px solid rgba(255, 255, 255, 0.1);
        background: rgba(255, 255, 255, 0.05);
        color: inherit;
        font-size: 16px;
        transition: border 0.3s;
    }

    .light-mode .auth-box input {
        border: 1px solid rgba(0, 0, 0, 0.1);
        background: rgba(0, 0, 0, 0.03);
    }

    .auth-box input:focus {
        outline: none;
        border-color: var(--accent);
    }

    .auth-box button {
        width: 100%;
        padding: 14px;
        border-radius: 10px;
        border: none;
        background: var(--accent);
        color: white;
        font-size: 16px;
        font-weight: 600;
        cursor: pointer;
        margin-top: 20px;
        transition: transform 0.3s;
    }

    .auth-box button:hover {
        transform: translateY(-2px);
    }

    .error {
        color: #ff6b6b;
        margin-top: 10px;
        padding: 10px;
        background: rgba(255, 107, 107, 0.1);
        border-radius: 8px;
    }

    .auth-links {
        margin-top: 20px;
        color: var(--muted);
    }

    .auth-links a {
        color: var(--accent);
        text-decoration: none;
        font-weight: 500;
    }

    .auth-links a:hover {
        text-decoration: underline;
    }

    .theme-toggle {
        position: absolute;
        top: 20px;
        right: 20px;
        background: none;
        border: none;
        font-size: 24px;
        cursor: pointer;
        color: inherit;
        padding: 8px;
        border-radius: 50%;
        transition: background 0.3s;
    }

    .theme-toggle:hover {
        background: rgba(255, 255, 255, 0.1);
    }

    .light-mode .theme-toggle:hover {
        background: rgba(0, 0, 0, 0.05);
    }
</style>
</head>
<body id="bodyRoot">
    <button class="theme-toggle" id="themeToggle">🌓</button>

    <div class="auth-container">
        <div class="auth-box">
            <h2>Welcome Back</h2>
            <p>Sign in to continue to SpendGenie</p>
            <form method="POST">
                <input name="email" type="email" placeholder="Email" required>
                <input name="password" type="password" placeholder="Password" required>
                <button type="submit">Sign In</button>
            </form>
            {% if error %}<div class="error">{{ error }}</div>{% endif %}
            <div class="auth-links">
                <p>New to SpendGenie? <a href="/register">Create an account</a></p>
                <p><a href="/">← Back to Home</a></p>
            </div>
        </div>
    </div>

    <script>
        // Theme Toggle for Auth Pages
        const themeToggle = document.getElementById('themeToggle');
        const body = document.getElementById('bodyRoot');

        // Check for saved theme preference
        if (localStorage.getItem('theme') === 'light') {
            body.classList.add('light-mode');
            themeToggle.textContent = '🌙';
        }

        themeToggle.addEventListener('click', () => {
            body.classList.toggle('light-mode');
            if (body.classList.contains('light-mode')) {
                localStorage.setItem('theme', 'light');
                themeToggle.textContent = '🌙';
            } else {
                localStorage.setItem('theme', 'dark');
                themeToggle.textContent = '🌓';
            }
        });
    </script>
</body></html>"""

register_html = """<!doctype html>
<html><head><meta charset="utf-8"><title>Register - SpendGenie</title>
<style>
    :root {
        --bg-dark: #0f0f0f;
        --bg-light: #f8f9fa;
        --card-dark: #1c1c1c;
        --card-light: #ffffff;
        --text-dark: #ffffff;
        --text-light: #333333;
        --accent: #0ea5ff;
        --muted: #b0b0b0;
    }

    * {
        margin: 0;
        padding: 0;
        box-sizing: border-box;
        font-family: 'Inter', sans-serif;
        transition: background-color 0.3s ease, color 0.3s ease, border-color 0.3s ease;
    }

    body {
        margin: 0;
        background: var(--bg-dark);
        color: var(--text-dark);
        min-height: 100vh;
        display: flex;
        align-items: center;
        justify-content: center;
        transition: background 0.3s, color 0.3s;
    }

    body.light-mode {
        background: var(--bg-light);
        color: var(--text-light);
    }

    .auth-container {
        width: 100%;
        max-width: 400px;
        padding: 20px;
    }

    .auth-box {
        background: var(--card-dark);
        padding: 40px;
        border-radius: 16px;
        text-align: center;
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.3);
        border: 1px solid rgba(255, 255, 255, 0.1);
    }

    .light-mode .auth-box {
        background: var(--card-light);
        border: 1px solid rgba(0, 0, 0, 0.1);
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
    }

    .auth-box h2 {
        font-size: 28px;
        margin-bottom: 10px;
        color: var(--accent);
    }

    .auth-box p {
        color: var(--muted);
        margin-bottom: 30px;
    }

    .light-mode .auth-box p {
        color: #666;
    }

    .auth-box input {
        width: 100%;
        padding: 14px;
        margin: 10px 0;
        border-radius: 10px;
        border: 1px solid rgba(255, 255, 255, 0.1);
        background: rgba(255, 255, 255, 0.05);
        color: inherit;
        font-size: 16px;
        transition: border 0.3s;
    }

    .light-mode .auth-box input {
        border: 1px solid rgba(0, 0, 0, 0.1);
        background: rgba(0, 0, 0, 0.03);
    }

    .auth-box input:focus {
        outline: none;
        border-color: var(--accent);
    }

    .auth-box button {
        width: 100%;
        padding: 14px;
        border-radius: 10px;
        border: none;
        background: var(--accent);
        color: white;
        font-size: 16px;
        font-weight: 600;
        cursor: pointer;
        margin-top: 20px;
        transition: transform 0.3s;
    }

    .auth-box button:hover {
        transform: translateY(-2px);
    }

    .error {
        color: #ff6b6b;
        margin-top: 10px;
        padding: 10px;
        background: rgba(255, 107, 107, 0.1);
        border-radius: 8px;
    }

    .auth-links {
        margin-top: 20px;
        color: var(--muted);
    }

    .auth-links a {
        color: var(--accent);
        text-decoration: none;
        font-weight: 500;
    }

    .auth-links a:hover {
        text-decoration: underline;
    }

    .theme-toggle {
        position: absolute;
        top: 20px;
        right: 20px;
        background: none;
        border: none;
        font-size: 24px;
        cursor: pointer;
        color: inherit;
        padding: 8px;
        border-radius: 50%;
        transition: background 0.3s;
    }

    .theme-toggle:hover {
        background: rgba(255, 255, 255, 0.1);
    }

    .light-mode .theme-toggle:hover {
        background: rgba(0, 0, 0, 0.05);
    }
</style>
</head>
<body id="bodyRoot">
    <button class="theme-toggle" id="themeToggle">🌓</button>

    <div class="auth-container">
        <div class="auth-box">
            <h2>Create Account</h2>
            <p>Join SpendGenie to start tracking your expenses</p>
            <form method="POST">
                <input name="username" placeholder="Username" required>
                <input name="email" type="email" placeholder="Email" required>
                <input name="password" type="password" placeholder="Password" required>
                <button type="submit">Create Account</button>
            </form>
            {% if error %}<div class="error">{{ error }}</div>{% endif %}
            <div class="auth-links">
                <p>Already have an account? <a href="/login">Sign in</a></p>
                <p><a href="/">← Back to Home</a></p>
            </div>
        </div>
    </div>

    <script>
        // Theme Toggle for Auth Pages
        const themeToggle = document.getElementById('themeToggle');
        const body = document.getElementById('bodyRoot');

        // Check for saved theme preference
        if (localStorage.getItem('theme') === 'light') {
            body.classList.add('light-mode');
            themeToggle.textContent = '🌙';
        }

        themeToggle.addEventListener('click', () => {
            body.classList.toggle('light-mode');
            if (body.classList.contains('light-mode')) {
                localStorage.setItem('theme', 'light');
                themeToggle.textContent = '🌙';
            } else {
                localStorage.setItem('theme', 'dark');
                themeToggle.textContent = '🌓';
            }
        });
    </script>
</body></html>"""

(TEMPLATE_DIR / "login.html").write_text(login_html)
(TEMPLATE_DIR / "register.html").write_text(register_html)

# ----------------- Routes -----------------
@app.route("/")
def landing_page():
    return render_template_string(landing_template)

@app.route("/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        email = request.form.get("email")
        pwd = request.form.get("password")
        user = User.query.filter_by(email=email).first()
        if user and bcrypt.check_password_hash(user.password, pwd):
            session["user_id"] = user.id
            session["session_id"] = str(uuid.uuid4())
            return redirect("/upload")
        else:
            return render_template("login.html", error="Invalid credentials")
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if request.method == "POST":
        username = request.form.get("username")
        email = request.form.get("email")
        pwd = request.form.get("password")
        if User.query.filter_by(email=email).first():
            return render_template("register.html", error="Email already used")
        hashed = bcrypt.generate_password_hash(pwd).decode("utf-8")
        u = User(username=username, email=email, password=hashed)
        db.session.add(u); db.session.commit()
        return redirect("/login")
    return render_template("register.html")

@app.route("/logout")
def logout():
    sid = session.get("session_id")
    if sid:
        d = UPLOAD_ROOT / sid
        if d.exists(): shutil.rmtree(d, ignore_errors=True)
    session.clear()
    return redirect("/")

@app.before_request
def require_login():
    path = request.path
    protected = ["/dashboard","/upload","/analysis","/summary","/years","/load_dashboard","/prediction","/expense_input","/profile","/chat", "/expense_tracker"]
    if any(path.startswith(p) for p in protected):
        if "user_id" not in session:
            return redirect("/login")

# ----------------- Expense Tracker Page (Simple UI) -----------------
@app.route("/expense_tracker", methods=["GET","POST"])
def expense_tracker():
    global main_df, available_categories, saved_expenses
    uploaded = False
    if request.method == "POST":
        file = request.files.get("file")
        if file:
            df = pd.read_csv(file)
            lower_map = {c.lower(): c for c in df.columns}
            mapped_item = lower_map.get("item") or lower_map.get("category") or df.columns[0]
            mapped_price = lower_map.get("price") or lower_map.get("amount") or (df.columns[1] if len(df.columns)>1 else mapped_item)
            mapped_date = lower_map.get("date") or (df.columns[2] if len(df.columns)>2 else None)

            newdf = pd.DataFrame()
            # item
            try:
                newdf["item"] = df[mapped_item].astype(str)
            except:
                newdf["item"] = df.iloc[:,0].astype(str)
            # price
            try:
                newdf["price"] = pd.to_numeric(df[mapped_price], errors="coerce").fillna(0.0)
            except:
                newdf["price"] = 0.0
            # date
            if mapped_date and mapped_date in df.columns:
                try:
                    newdf["date"] = pd.to_datetime(df[mapped_date], errors="coerce")
                except:
                    newdf["date"] = pd.NaT
            else:
                newdf["date"] = pd.NaT
            newdf["description"] = ""
            main_df = pd.concat([main_df, newdf], ignore_index=True, sort=False)
            normalize_main_df()
            uploaded = True
            return redirect("/prediction")

    totals = totals_by_category(period="all")
    total_all = total_expense(period="all")
    view = request.args.get("view","all")
    content = f"""
        <h1>Expense Tracker</h1>

        <div class="upload-box">
            <form method="POST" enctype="multipart/form-data">
                <input type="file" name="file" accept=".csv" required>
                <button type="submit">Upload CSV</button>
            </form>
            {f'<div style="margin-top:10px;color:#0ea5ff;">CSV uploaded successfully — categories loaded.</div>' if uploaded else ''}
        </div>

        <div style="margin-top:18px;display:flex;align-items:center;gap:12px;">
            <div class="card" style="padding:12px 18px;">
                <strong>Total:</strong> ₹{total_all:.2f}
            </div>

            <form method="GET" action="/expense_tracker" style="display:inline-block;">
                <label style="margin-left:8px;color:#bbb;">View:</label>
                <select name="view" onchange="this.form.submit()" style="margin-left:8px;padding:6px;border-radius:6px;background:#111;color:white;border:1px solid #333;">
                    <option value="all" {'selected' if view=='all' else ''}>All</option>
                    <option value="daily" {'selected' if view=='daily' else ''}>Daily</option>
                    <option value="weekly" {'selected' if view=='weekly' else ''}>Weekly</option>
                    <option value="monthly" {'selected' if view=='monthly' else ''}>Monthly</option>
                </select>
            </form>
        </div>

        {f'''
        <h2 style="margin-top:20px;">Categories</h2>
        <div class="grid">
            {''.join([f'''
            <div class="card">
                <div class="cat-title">{cat}</div>
                <div class="cat-amount">₹{amt:.2f}</div>
            </div>''' for cat, amt in totals.items()])}
        </div>
        ''' if totals else ''}
    """
    return render_template_string(app_base_template, content=content, active_page="upload", page_title="Expense Tracker")

# ----------------- Main Routes with Sidebar Layout -----------------
@app.route("/upload", methods=["GET","POST"])
def upload():
    preview_html = None
    message = None
    if request.method == "POST":
        f = request.files.get("file")
        if f and f.filename.lower().endswith(".csv"):
            try:
                path = save_uploaded_csv(f)
                dfp = pd.read_csv(path, nrows=10)
                preview_html = dfp.to_html(classes="preview", index=False)
            except Exception as e:
                message = f"Preview failed: {e}"
        else:
            message = "Please upload a CSV file."

    content = f"""
    <h1 style="margin-bottom: 20px;">Upload CSV</h1>
    <p style="color: var(--muted); margin-bottom: 30px;">Upload your expense data to start analyzing and predicting.</p>

    <div class="upload-box">
        <form method="POST" enctype="multipart/form-data">
            <input type="file" name="file" accept=".csv" required style="margin-bottom: 20px;">
            <button type="submit" style="padding: 12px 24px;">Upload & Preview</button>
        </form>
    </div>

    {f'<div style="margin-top: 30px;"><h3>Preview (first 10 rows)</h3>{preview_html}<form method="POST" action="/load_dashboard" style="margin-top: 20px;"><button type="submit" style="padding: 12px 24px;">Load & Continue to Dashboard</button></form></div>' if preview_html else ''}
    {f'<p class="error" style="margin-top: 20px;">{message}</p>' if message else ''}

    <div class="card" style="margin-top: 30px;">
        <h3>📋 CSV Format Tips</h3>
        <p>Your CSV should include columns like:</p>
        <ul style="padding-left: 20px; margin-top: 10px;">
            <li><strong>Date</strong> - Transaction date</li>
            <li><strong>Amount</strong> - Transaction amount</li>
            <li><strong>Category</strong> - Expense category</li>
            <li><strong>Description</strong> - Transaction description (optional)</li>
        </ul>
    </div>
    """
    return render_template_string(app_base_template, content=content, active_page="upload", page_title="Upload CSV")

@app.route("/load_dashboard", methods=["POST"])
def load_dashboard():
    p = get_user_csv_path()
    if not p:
        content = "<p class='error'>No CSV available. Upload a CSV first.</p>"
        return render_template_string(app_base_template, content=content, active_page="upload", page_title="Upload CSV")
    return redirect("/dashboard")

@app.route("/dashboard")
def dashboard():
    content = """
    <style>
        select, option {
            color: #000 !important;
            background-color: #ffffff !important;
            font-weight: 600;
            padding: 10px;
            border-radius: 8px;
            border: 2px solid #0ea5ff;
            cursor: pointer;
        }

        @media (prefers-color-scheme: dark) {
            select, option {
                color: #ffffff !important;
                background-color: #222 !important;
                border: 2px solid #38bdf8;
            }
        }

        select:hover {
            border-color: #38bdf8;
            box-shadow: 0 0 6px rgba(56,189,248,0.5);
        }

        /* Better spacing */
        .filters select, .filters input, .filters button {
            margin-right: 10px;
        }
    </style>
    <h1 style="margin-bottom: 20px;">Dashboard</h1>
    <p style="color: var(--muted); margin-bottom: 30px;">Analyze your spending patterns with interactive charts and filters.</p>

    <section class="filters">
        <select id="year" onchange="applyFilters()"><option value=''>All Years</option></select>
        <select id="month" onchange="applyFilters()"><option value=''>All Months</option>
          <option value="1">Jan</option><option value="2">Feb</option><option value="3">Mar</option>
          <option value="4">Apr</option><option value="5">May</option><option value="6">Jun</option>
          <option value="7">Jul</option><option value="8">Aug</option><option value="9">Sep</option>
          <option value="10">Oct</option><option value="11">Nov</option><option value="12">Dec</option>
        </select>
        <input id="start" type="date" onchange="applyFilters()">
        <input id="end" type="date" onchange="applyFilters()">
        <button onclick="generateSummary()">AI Summary</button>
    </section>

    <section id="aiSummary" class="ai-summary">AI summary will appear here. Click "AI Summary" to generate insights.</section>

    <section class="grid">
        <div class="card"><h3>📅 Monthly Trends</h3><div id="monthly"></div></div>
        <div class="card"><h3>📊 Category Breakdown</h3><div id="category"></div></div>
        <div class="card"><h3>📈 Peak Spending Days</h3><div id="peak"></div></div>
        <div class="card"><h3>📋 Yearly Comparison</h3><div id="yearly"></div></div>
    </section>

    <script>
    async function loadYears(){
        const res = await fetch("/years");
        const years = await res.json();
        const sel = document.getElementById("year");
        sel.innerHTML = "<option value=''>All Years</option>" + years.map(y=>`<option value="${y}">${y}</option>`).join("");
    }

    async function applyFilters(){
        const filters = {
            year: document.getElementById("year").value || "",
            month: document.getElementById("month").value || "",
            start: document.getElementById("start").value || "",
            end: document.getElementById("end").value || ""
        };
        const params = new URLSearchParams(filters);
        const res = await fetch("/analysis?"+params.toString());
        const data = await res.json();
        updateCharts(data);
        updateAISummary(data.summary);
    }

    function updateCharts(data){
        if(data.monthly && data.monthly.length > 0){
            Plotly.newPlot('monthly', [{
                x: data.monthly.map(r=>r.month_year),
                y: data.monthly.map(r=>r.amount),
                type: 'bar', marker:{color:'#0ea5ff'}
            }], {margin:{t:20}, plot_bgcolor: 'transparent', paper_bgcolor: 'transparent'});
        } else {
            document.getElementById('monthly').innerHTML = '<div class="empty">No monthly data available</div>';
        }

        if(data.category && data.category.length > 0){
            Plotly.newPlot('category', [{
                labels: data.category.map(r=>r.category),
                values: data.category.map(r=>r.amount),
                type:'pie', textinfo:'label+percent'
            }], {margin:{t:20}, plot_bgcolor: 'transparent', paper_bgcolor: 'transparent'});
        } else {
            document.getElementById('category').innerHTML = '<div class="empty">No category data available</div>';
        }

        if(data.peak && data.peak.length > 0 && !(data.peak.length===1 && data.peak[0].day==="No Data")){
            const x = data.peak.map(r=>r.day || r.date);
            const y = data.peak.map(r=>r.amount);
            Plotly.newPlot('peak', [{x,y,mode:'lines+markers',line:{color:'#ff4d4d'}}], {margin:{t:20}, plot_bgcolor: 'transparent', paper_bgcolor: 'transparent'});
        } else {
            document.getElementById('peak').innerHTML = '<div class="empty">No peak spending data</div>';
        }

        if(data.yearly && data.yearly.length > 0){
            Plotly.newPlot('yearly', [{
                x: data.yearly.map(r=>r.year),
                y: data.yearly.map(r=>r.amount),
                type:'scatter', mode:'lines+markers', line:{color:'#22bb88'}
            }], {margin:{t:20}, plot_bgcolor: 'transparent', paper_bgcolor: 'transparent'});
        } else {
            document.getElementById('yearly').innerHTML = '<div class="empty">No yearly data available</div>';
        }
    }

    function updateAISummary(summary){
        const el = document.getElementById('aiSummary');
        if (!summary) { el.innerText = "AI summary will appear here. Click 'AI Summary' to generate insights."; return; }
        el.innerHTML = `
            <div style="text-align: left; padding: 10px;">
                <h4 style="margin-bottom: 10px; color: #0ea5ff;">🧠 AI Summary</h4>
                <p><strong>Total Spending:</strong> ₹${Number(summary.total||0).toFixed(2)}</p>
                <p><strong>Top Category:</strong> ${summary.top_cat || '-'}</p>
                <p><strong>Peak Spending Day:</strong> ${summary.peak_day || '-'}</p>
                ${summary.peak_amount ? `<p><strong>Peak Amount:</strong> ₹${Number(summary.peak_amount||0).toFixed(2)}</p>` : ''}
            </div>
        `;
    }

    async function generateSummary(){
        const filters = {
            year: document.getElementById("year").value || "",
            month: document.getElementById("month").value || "",
            start: document.getElementById("start").value || "",
            end: document.getElementById("end").value || ""
        };
        const res = await fetch("/summary?"+ new URLSearchParams(filters));
        const text = await res.text();
        document.getElementById("aiSummary").innerHTML = `
            <div style="text-align: left; padding: 10px;">
                <h4 style="margin-bottom: 10px; color: #0ea5ff;">🧠 AI Generated Summary</h4>
                <div style="white-space: pre-line; line-height: 1.6;">${text}</div>
            </div>
        `;
    }

    loadYears();
    applyFilters();
    </script>
    """
    return render_template_string(app_base_template, content=content, active_page="dashboard", page_title="Dashboard")

@app.route("/prediction")
def prediction():
    try:
        # Load correct CSV
        csv_path = get_user_csv_path()
        if not csv_path:
            raise ValueError("No CSV found. Please upload a dataset first.")

        df, meta = load_and_prepare(csv_path)

        # Ensure cleaned column names match prediction logic
        df = df.rename(columns={
            meta["date_col"]: "date",
            meta["cat_col"]: "category" if meta["cat_col"] else "Category",
            "expense": "amount"
        })

        df["date"] = pd.to_datetime(df["date"], errors="coerce")
        expenses = df[df["amount"] > 0].copy()

        if len(expenses) == 0:
            content = """
            <h1>AI Prediction</h1>
            <div class='card'>
                <h3>⚠ No Expense Data Found</h3>
                <p>Please upload a valid CSV file with expense entries.</p>
                <a href='/upload'><button>Upload CSV</button></a>
            </div>
            """
            return render_template_string(app_base_template, content=content, active_page="prediction", page_title="Prediction")

        # --- Prediction Engine ---
        past_months_df = aggregate_past_months(expenses)
        past_labels = past_months_df['label'].tolist()
        past_totals = list(past_months_df['amount'].round(2))

        total_next = predict_next_total(past_months_df)
        category_predictions = predict_next_by_category(expenses)

        category_names = list(category_predictions.keys())
        category_values = list(category_predictions.values())

        # --- UI Rendering ---
        content = f"""
        <h1 style="margin-bottom: 20px;">AI Expense Forecast</h1>
        <p style="color: var(--muted); margin-bottom: 30px;">Machine learning-based prediction using historical spending patterns.</p>

        <div class="card" style="display:flex; justify-content:space-between; padding:25px;">
            <div>
                <h3>📅 Next Month Prediction</h3>
                <div style="font-size: 42px; font-weight: bold; color: #0ea5ff; margin-top: 20px;">₹{total_next}</div>
                <p style="font-size:14px; opacity:0.7">(Regression powered estimate)</p>
            </div>
            <div style="text-align:right;">
                <h3>📈 Model Note</h3>
                <p style="font-size:14px; opacity:0.7">Accuracy improves as more months are added.</p>
            </div>
        </div>

        <div style="display:grid; grid-template-columns:1fr 1fr; gap:20px; margin-top:20px;">
            <div class="card">
                <h3>📊 Monthly Trend</h3>
                <div id="forecastTrend"></div>
            </div>

            <div class="card">
                <h3>📂 Category Forecast</h3>
                <div id="forecastCategory"></div>
            </div>
        </div>

        <div class="card" style="margin-top:20px;">
            <h3>📜 Category Breakdown</h3>
            {'<br>'.join([f'<strong>{cat}:</strong> ₹{val}' for cat,val in category_predictions.items()])}
        </div>

        <script>
        Plotly.newPlot('forecastTrend', [{{
            x: {json.dumps(past_labels + ["Next Month"])},
            y: {json.dumps(past_totals + [total_next])},
            type:'scatter',
            mode:'lines+markers',
            line:{{color:'#0ea5ff', width:3}}
        }}],{{margin:{{t:20}}}});

        Plotly.newPlot('forecastCategory', [{{
            x: {json.dumps(category_names)},
            y: {json.dumps(category_values)},
            type:'bar',
            marker:{{color:'#6f42ff'}}
        }}],{{margin:{{t:20}}}});

        </script>
        """

    except Exception as e:
        content = f"""
        <h1>Prediction Error</h1>
        <div class='card'>
            <h3>⚠ Failed to Generate Prediction</h3>
            <p>{str(e)}</p>
            <a href='/upload'><button>Re-upload CSV</button></a>
        </div>
        """

    return render_template_string(app_base_template, content=content, active_page="prediction", page_title="Prediction")

@app.route("/expense_input", methods=["GET", "POST"])
def expense_input():
    global main_df, saved_expenses, available_categories
    normalize_main_df()

    # Enhanced CSS for this page
    enhanced_css = """
    <style>
        .expense-form-container {
            max-width: 500px;
            margin: 0 auto;
        }

        .input-group {
            margin-bottom: 24px;
            position: relative;
        }

        .input-group label {
            display: block;
            margin-bottom: 8px;
            color: var(--accent);
            font-weight: 600;
            font-size: 14px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }

        .form-input {
            width: 100%;
            padding: 14px 16px;
            border-radius: 12px;
            border: 2px solid var(--border-dark);
            background: rgba(255, 255, 255, 0.05);
            color: inherit;
            font-size: 16px;
            transition: all 0.3s ease;
        }

        .light-mode .form-input {
            background: rgba(0, 0, 0, 0.02);
            border-color: var(--border-light);
        }

        .form-input:focus {
            outline: none;
            border-color: var(--accent);
            box-shadow: 0 0 0 3px rgba(14, 165, 255, 0.2);
            transform: translateY(-1px);
        }

        .form-input::placeholder {
            color: var(--muted);
            opacity: 0.7;
        }

        .select-wrapper {
            position: relative;
        }

        .select-wrapper::after {
            content: "▼";
            position: absolute;
            right: 16px;
            top: 50%;
            transform: translateY(-50%);
            color: var(--accent);
            pointer-events: none;
            font-size: 12px;
        }

        .category-select {
            appearance: none;
            cursor: pointer;
            padding-right: 40px;
        }

        .amount-input-wrapper {
            position: relative;
        }

        .currency-symbol {
            position: absolute;
            left: 16px;
            top: 50%;
            transform: translateY(-50%);
            color: var(--accent);
            font-weight: bold;
            font-size: 18px;
        }

        .amount-input {
            padding-left: 40px;
        }

        .description-textarea {
            min-height: 100px;
            resize: vertical;
            font-family: inherit;
            line-height: 1.5;
        }

        .button-group {
            display: flex;
            gap: 12px;
            margin-top: 32px;
        }

        .primary-button {
            flex: 1;
            padding: 16px 24px;
            background: linear-gradient(135deg, #0ea5ff, #5a2aff);
            color: white;
            border: none;
            border-radius: 12px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s ease;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 8px;
        }

        .primary-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 8px 25px rgba(90, 42, 255, 0.3);
            background: linear-gradient(135deg, #2ab1ff, #6a3aff);
        }

        .primary-button:active {
            transform: translateY(0);
        }

        .secondary-button {
            flex: 1;
            padding: 16px 24px;
            background: rgba(255, 255, 255, 0.1);
            color: inherit;
            border: 2px solid var(--accent);
            border-radius: 12px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s ease;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 8px;
        }

        .light-mode .secondary-button {
            background: rgba(0, 0, 0, 0.02);
        }

        .secondary-button:hover {
            background: rgba(14, 165, 255, 0.15);
            transform: translateY(-2px);
        }

        .success-message {
            margin-top: 20px;
            padding: 16px;
            background: linear-gradient(135deg, rgba(46, 204, 113, 0.2), rgba(46, 204, 113, 0.1));
            border: 1px solid rgba(46, 204, 113, 0.3);
            border-radius: 12px;
            color: #2ecc71;
            display: flex;
            align-items: center;
            gap: 10px;
            animation: slideIn 0.3s ease;
        }

        .light-mode .success-message {
            background: linear-gradient(135deg, rgba(46, 204, 113, 0.1), rgba(46, 204, 113, 0.05));
        }

        .empty-state {
            text-align: center;
            padding: 40px 20px;
            background: rgba(14, 165, 255, 0.05);
            border-radius: 16px;
            margin-bottom: 30px;
            border: 2px dashed rgba(14, 165, 255, 0.3);
        }

        .light-mode .empty-state {
            background: rgba(14, 165, 255, 0.02);
        }

        .empty-state-icon {
            font-size: 48px;
            margin-bottom: 16px;
            opacity: 0.7;
        }

        .recent-expenses {
            margin-top: 40px;
        }

        .expense-card {
            background: rgba(255, 255, 255, 0.05);
            border-radius: 12px;
            padding: 16px;
            margin-bottom: 12px;
            border-left: 4px solid var(--accent);
            transition: transform 0.2s ease;
        }

        .light-mode .expense-card {
            background: rgba(0, 0, 0, 0.02);
            border-left: 4px solid var(--accent);
        }

        .expense-card:hover {
            transform: translateX(4px);
            background: rgba(14, 165, 255, 0.1);
        }

        .expense-header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 8px;
        }

        .expense-category {
            font-weight: 600;
            color: var(--accent);
            font-size: 16px;
        }

        .expense-amount {
            font-weight: 700;
            font-size: 18px;
            color: #ff4f8c;
        }

        .expense-date {
            color: var(--muted);
            font-size: 14px;
            margin-bottom: 8px;
        }

        .expense-description {
            color: inherit;
            font-size: 14px;
            line-height: 1.5;
        }

        @keyframes slideIn {
            from {
                opacity: 0;
                transform: translateY(-10px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }

        @media (max-width: 768px) {
            .button-group {
                flex-direction: column;
            }

            .primary-button,
            .secondary-button {
                width: 100%;
            }
        }
    </style>
    """

    success = False
    if request.method == "POST":
        cat = request.form.get("category")
        amt = request.form.get("amount")
        date_str = request.form.get("date")
        desc = request.form.get("description", "")

        try:
            amt_val = float(amt)
        except:
            amt_val = 0.0

        try:
            date_val = pd.to_datetime(date_str)
        except:
            date_val = pd.Timestamp(datetime.now())

        saved_expenses.append({
            "category": cat,
            "amount": amt_val,
            "date": date_val.strftime("%Y-%m-%d"),
            "description": desc
        })

        # Add to main dataframe if category doesn't exist
        if cat not in available_categories:
            available_categories.append(cat)

        newrow = {
            "item": cat,
            "price": amt_val,
            "date": date_val,
            "description": desc
        }

        main_df = pd.concat([main_df, pd.DataFrame([newrow])], ignore_index=True)
        normalize_main_df()
        success = True

    if not available_categories:
        content = f"""
        {enhanced_css}
        <div class="expense-form-container">
            <h1 style="margin-bottom: 10px; background: linear-gradient(135deg, #0ea5ff, #ff4f8c); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text;">➕ Add New Expense</h1>
            <p style="color: var(--muted); margin-bottom: 30px;">Start tracking your expenses by adding your first category.</p>

            <div class="empty-state">
                <div class="empty-state-icon">📋</div>
                <h3 style="margin-bottom: 10px;">No Categories Found</h3>
                <p style="color: var(--muted); margin-bottom: 20px;">Create your first expense category to get started with tracking.</p>
            </div>

            <form method="POST">
                <div class="input-group">
                    <label for="category">NEW CATEGORY</label>
                    <input type="text"
                           name="category"
                           class="form-input"
                           placeholder="Enter category name (e.g., Food, Rent, Transportation)"
                           required>
                </div>

                <div class="input-group">
                    <label for="amount">AMOUNT</label>
                    <div class="amount-input-wrapper">
                        <span class="currency-symbol">₹</span>
                        <input type="number"
                               name="amount"
                               step="0.01"
                               class="form-input amount-input"
                               placeholder="Enter amount"
                               required>
                    </div>
                </div>

                <div class="input-group">
                    <label for="date">DATE</label>
                    <input type="date"
                           name="date"
                           class="form-input"
                           required>
                </div>

                <div class="input-group">
                    <label for="description">DESCRIPTION (OPTIONAL)</label>
                    <textarea name="description"
                              class="form-input description-textarea"
                              placeholder="Add details about this expense..."></textarea>
                </div>

                <div class="button-group">
                    <button type="submit" class="primary-button">
                        <span>💾</span> Save Expense
                    </button>
                    <a href="/upload" style="text-decoration: none; flex: 1;">
                        <button type="button" class="secondary-button">
                            <span>📁</span> Upload CSV First
                        </button>
                    </a>
                </div>
            </form>

            {f'<div class="success-message"><span>✅</span> Expense added successfully! You can now add more expenses in this category.</div>' if success else ''}
        </div>
        """
    else:
        content = f"""
        {enhanced_css}
        <div class="expense-form-container">
            <h1 style="margin-bottom: 10px; background: linear-gradient(135deg, #0ea5ff, #ff4f8c); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text;">➕ Add Manual Expense</h1>
            <p style="color: var(--muted); margin-bottom: 30px;">Quickly add expenses to your tracking data.</p>

            <form method="POST">
                <div class="input-group">
                    <label for="category">CATEGORY</label>
                    <div class="select-wrapper">
                        <select name="category" class="form-input category-select" required>
                            <option value="" disabled selected>Select a category...</option>
                            {''.join([f'<option value="{c}">{c}</option>' for c in available_categories])}
                        </select>
                    </div>
                </div>

                <div class="input-group">
                    <label for="amount">AMOUNT</label>
                    <div class="amount-input-wrapper">
                        <span class="currency-symbol">₹</span>
                        <input type="number"
                               name="amount"
                               step="0.01"
                               class="form-input amount-input"
                               placeholder="0.00"
                               required>
                    </div>
                </div>

                <div class="input-group">
                    <label for="date">DATE</label>
                    <input type="date"
                           name="date"
                           class="form-input"
                           value="{datetime.now().strftime('%Y-%m-%d')}"
                           required>
                </div>

                <div class="input-group">
                    <label for="description">DESCRIPTION (OPTIONAL)</label>
                    <textarea name="description"
                              class="form-input description-textarea"
                              placeholder="Add any details about this expense..."></textarea>
                </div>

                <div class="button-group">
                    <button type="submit" class="primary-button">
                        <span>💾</span> Save Expense
                    </button>
                    <a href="/upload" style="text-decoration: none; flex: 1;">
                        <button type="button" class="secondary-button">
                            <span>📁</span> Upload CSV
                        </button>
                    </a>
                </div>
            </form>

            {f'<div class="success-message"><span>✅</span> Expense added successfully!</div>' if success else ''}

            {f'''
            <div class="recent-expenses">
                <h3 style="margin-bottom: 20px;">📝 Recent Expenses</h3>
                {'<p style="color: var(--muted); text-align: center;">No recent expenses yet.</p>' if not saved_expenses else ''.join([f'''
                <div class="expense-card">
                    <div class="expense-header">
                        <div class="expense-category">{e["category"]}</div>
                        <div class="expense-amount">₹{e["amount"]:.2f}</div>
                    </div>
                    <div class="expense-date">{e["date"]}</div>
                    {f'<div class="expense-description">{e["description"]}</div>' if e["description"] else ''}
                </div>
                ''' for e in saved_expenses[-5:]])}
            </div>
            ''' if saved_expenses else ''}
        </div>
        """

    return render_template_string(app_base_template, content=content, active_page="expense_input", page_title="Add Expense")

@app.route("/profile")
def profile():
    user = User.query.get(session.get("user_id")) if session.get("user_id") else None

    if user:
        content = f"""
        <h1 style="margin-bottom: 20px;">User Profile</h1>
        <p style="color: var(--muted); margin-bottom: 30px;">Manage your account and data.</p>

        <div class="card" style="max-width: 600px;">
            <h3>👤 Account Information</h3>
            <div style="margin-top: 20px; padding: 20px; background: rgba(14, 165, 255, 0.05); border-radius: 10px;">
                <p><strong>Username:</strong> {user.username}</p>
                <p><strong>Email:</strong> {user.email}</p>
                <p><strong>Account Type:</strong> Standard User</p>
            </div>

            <h3 style="margin-top: 30px;">📊 Data Management</h3>
            <p>Your uploaded data is stored securely and privately.</p>
            <div style="margin-top: 20px; display: flex; gap: 10px;">
                <a href="/upload"><button>📁 Upload New Data</button></a>
                <a href="/dashboard"><button>📊 View Dashboard</button></a>
            </div>
        </div>
        """
    else:
        content = """
        <h1 style="margin-bottom: 20px;">User Profile</h1>
        <div class="card">
            <h3>🔒 Not Logged In</h3>
            <p>Please log in to view your profile.</p>
            <a href="/login"><button style="margin-top: 20px;">Go to Login</button></a>
        </div>
        """

    return render_template_string(app_base_template, content=content, active_page="profile", page_title="Profile")

# ----------------- API Routes -----------------
@app.route("/years")
def years_api():
    path = get_user_csv_path()
    if not path:
        return jsonify([])
    try:
        df, meta = load_and_prepare(path)
    except Exception:
        return jsonify([])
    yrs = sorted(df["year"].dropna().unique().tolist())
    return jsonify(yrs)

@app.route("/analysis")
def analysis_api():
    path = get_user_csv_path()
    if not path:
        return jsonify({"monthly": [], "category": [], "peak": [{"day":"No Data","amount":0}], "yearly": [], "summary": {}})
    try:
        df, meta = load_and_prepare(path)
    except Exception as e:
        return jsonify({"monthly": [], "category": [], "peak": [{"day":"No Data","amount":0}], "yearly": [], "summary": {}, "error": str(e)})

    year = request.args.get("year")
    month = request.args.get("month")
    start = request.args.get("start")
    end = request.args.get("end")

    temp = df.copy()
    if year:
        try: temp = temp[temp["year"] == int(year)]
        except: pass
    if month:
        try: temp = temp[temp["month"] == int(month)]
        except: pass
    if start:
        try: temp = temp[temp[meta["date_col"]] >= pd.to_datetime(start)]
        except: pass
    if end:
        try: temp = temp[temp[meta["date_col"]] <= pd.to_datetime(end)]
        except: pass

    expenses = temp[temp["expense"] > 0].copy()

    monthly = expenses.groupby("month_year")["expense"].sum().reset_index().rename(columns={"expense":"amount"})
    category = expenses.groupby("category_clean")["expense"].sum().reset_index().rename(columns={"category_clean":"category","expense":"amount"}).sort_values("amount", ascending=False)
    peak_data = expenses.groupby("day")["expense"].sum()
    if len(peak_data) > 0:
        peak = peak_data.reset_index(name="amount").sort_values("amount", ascending=False).head(10)
    else:
        peak = pd.DataFrame({"day":["No Data"], "amount":[0]})
    yearly = expenses.groupby("year")["expense"].sum().reset_index().rename(columns={"expense":"amount"})

    total = float(expenses["expense"].sum()) if len(expenses)>0 else 0.0
    top_cat = category.iloc[0]["category"] if len(category)>0 else None
    peak_row = peak.iloc[0] if len(peak)>0 else None
    summary = {
        "total": total,
        "top_cat": top_cat,
        "peak_day": str(peak_row["day"]) if peak_row is not None else None,
        "peak_amount": float(peak_row["amount"]) if peak_row is not None else 0.0
    }

    return jsonify({
        "monthly": monthly.to_dict(orient="records"),
        "category": category.to_dict(orient="records"),
        "peak": peak.to_dict(orient="records"),
        "yearly": yearly.to_dict(orient="records"),
        "summary": summary
    })

@app.route("/summary")
def summary_api():
    path = get_user_csv_path()
    if not path:
        return "No dataset uploaded."
    try:
        df, meta = load_and_prepare(path)
    except Exception as e:
        return "Failed to load dataset: " + str(e)

    year = request.args.get("year")
    month = request.args.get("month")
    start = request.args.get("start")
    end = request.args.get("end")
    temp = df.copy()
    if year:
        try: temp = temp[temp["year"] == int(year)]
        except: pass
    if month:
        try: temp = temp[temp["month"] == int(month)]
        except: pass
    if start:
        try: temp = temp[temp[meta["date_col"]] >= pd.to_datetime(start)]
        except: pass
    if end:
        try: temp = temp[temp[meta["date_col"]] <= pd.to_datetime(end)]
        except: pass

    expenses = temp[temp["expense"] > 0]
    if len(expenses)==0:
        return "No data available for the selected period."
    total = float(expenses["expense"].sum())
    cat_sums = expenses.groupby("category_clean")["expense"].sum().sort_values(ascending=False)
    top_cat = cat_sums.index[0]
    top_amt = cat_sums.iloc[0]
    insight_lines = [
        f"🧠 AI Summary (filtered range)",
        f"• Total spending: ₹{total:,.2f}",
        f"• Top category: {top_cat} (₹{top_amt:,.2f})",
    ]
    try:
        last_month = expenses["month_year"].max()
        prev = expenses[expenses["month_year"] < last_month].groupby("month_year")["expense"].sum()
        last_val = expenses[expenses["month_year"]==last_month]["expense"].sum()
        prev_mean = prev.mean() if len(prev)>0 else 0
        if prev_mean>0:
            pct = (last_val - prev_mean)/prev_mean*100
            if pct > 10:
                insight_lines.append(f"• Spending increased ~{pct:.0f}% vs previous months — review {top_cat}.")
            elif pct < -10:
                insight_lines.append("• Spending decreased compared with previous months — good job!")
    except Exception:
        pass
    insight_lines.append("• Suggestion: set a limit for the top category to reduce recurring spend.")
    return "\n".join(insight_lines)

@app.route("/chat", methods=["POST"])
def chat_api():
    data = request.get_json(force=True)
    msg = (data.get("message") or "").lower().strip()
    reply = "Sorry, I couldn't understand. Try: 'top categories', 'monthly trend', 'reduce dining', or 'save more'."

    # If dataset exists, compute some quick stats
    path = get_user_csv_path()
    df = None
    try:
        if path:
            df, meta = load_and_prepare(path)
    except Exception:
        df = None

    # Quick rule-based responses
    if "top category" in msg or "top categories" in msg or "top spend" in msg:
        if df is None:
            reply = "I don't see a dataset yet. Upload your CSV at Upload → Load & Continue to Dashboard."
        else:
            cat_sums = df.groupby("category_clean")["expense"].sum().sort_values(ascending=False)
            top = cat_sums.head(3).to_dict()
            lines = ["Top spending categories:"]
            for k,v in top.items():
                lines.append(f"• {k}: ₹{v:,.2f}")
            reply = "\n".join(lines)
    elif "monthly" in msg or "trend" in msg:
        if df is None:
            reply = "No data yet — upload a CSV to get monthly trends."
        else:
            monthly = df.groupby("month_year")["expense"].sum().sort_index()
            if len(monthly)==0:
                reply = "No expense data found in the file."
            else:
                last = monthly.iloc[-1]
                mean_prev = monthly.iloc[:-1].mean() if len(monthly)>1 else monthly.iloc[-1]
                if mean_prev>0:
                    pct = (last-mean_prev)/mean_prev*100
                    reply = f"Latest month total: ₹{last:,.2f}. That's {pct:+.0f}% vs average of prior months."
                else:
                    reply = f"Latest month total: ₹{last:,.2f}."
    elif "reduce" in msg or "save" in msg or "cut" in msg:
        # Generic saving tips + personalised if df present
        tips = [
            "Track subscriptions and cancel unused ones.",
            "Set a weekly dining out limit and carry cash for it.",
            "Automate a small transfer to savings each payday."
        ]
        if df is not None:
            cat_sums = df.groupby("category_clean")["expense"].sum().sort_values(ascending=False)
            if len(cat_sums)>0:
                top = cat_sums.index[0]
                tips.insert(0, f"You spend most on '{top}'. Consider a spending cap or alternative cheaper choices for this category.")
        reply = "Suggestions:\n• " + "\n• ".join(tips)
    elif "prediction" in msg or "forecast" in msg:
        if df is None:
            reply = "Upload your CSV data to get expense predictions."
        else:
            try:
                # Use prediction functions
                df['date'] = pd.to_datetime(df[meta['date_col']])
                df['amount'] = df['amount_clean']
                monthly_totals = aggregate_past_months(df)
                predicted_next = predict_next_total(monthly_totals)
                reply = f"Predicted spending for next month: ₹{predicted_next:.2f}"
            except Exception as e:
                reply = f"Could not generate prediction: {str(e)}"
    elif "hello" in msg or "hi" in msg or "hey" in msg:
        reply = "Hi! I'm SpendGenie AI. Ask me about 'top categories', 'monthly trend', 'predictions', or say 'reduce dining' to get tips."
    elif "help" in msg:
        reply = "I can help with:\n• Analyzing your spending patterns\n• Predicting future expenses\n• Budgeting tips\n• Understanding your top categories\nTry asking: 'What are my top categories?' or 'How can I save money?'"
    else:
        # fallback: attempt to answer using simple keywords
        if "dining" in msg or "food" in msg or "restaurants" in msg:
            reply = "Dining tips: set a weekly limit, prefer home-cooked meals, and track small purchases — they add up."
        elif "subscription" in msg or "subscriptions" in msg:
            reply = "Check your recurring charges: map them and cancel ones you rarely use."
        elif "budget" in msg:
            reply = "Budgeting tip: Use the 50/30/20 rule - 50% needs, 30% wants, 20% savings."
        else:
            # last fallback using dataset summary
            if df is not None:
                total = df["expense"].sum()
                reply = f"I analyzed your data: total recorded spending: ₹{total:,.2f}. Ask 'top categories' or 'monthly trend' for more details."
            else:
                reply = "I can provide budget tips and analyze uploaded CSVs. Upload a CSV to get personalized suggestions."

    return jsonify({"reply": reply})

# ----------------- Start ngrok and run -----------------
ngrok.set_auth_token(NGROK_TOKEN)
try: ngrok.kill()
except: pass

public_url = ngrok.connect(5000).public_url
print("\n" + "="*60)
print("🚀 SPENDGENIE - AI Expense Tracker")
print("="*60)
print(f"🌍 Public URL: {public_url}")
print("="*60 + "\n")

# Run the app
if __name__ == '__main__':
    app.run(port=5000, debug=False, use_reloader=False)

ERROR: Operation cancelled by user


KeyboardInterrupt: 